# Impute missing values using kNN
**For preprocessing – imputing missing values**. Output of this is fed into the NMF model. See **Fig. 1** in manuscript.

In [1]:
import numpy as np
import pickle
import sklearn
#from nmf_with_missing_values import nmf_with_missing_values
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsRegressor as kNN
from tqdm import tqdm
%matplotlib inline

## load the data

In [2]:
#load_data
tmp = np.load('../data/mouse_brain_ISH_float32.npz',allow_pickle=True)
data = tmp['data']
sections = tmp['sections'].tolist()
original_shape = data.shape

In [3]:
data.shape

(4345, 67, 41, 58)

### define kNN method to do imputation

In [4]:
#define_model

class kNN_imputation:
    def __init__(self, 
                 n_neighbors = 1,
                 weights = 'uniform',
                 metric = 'euclidean'):
        ''' Init function of kNN_imputation
        
        '''
        self.model = kNN(n_neighbors=n_neighbors, metric=metric, weights = weights, n_jobs=4)
    def fit_transform(self, X, template, inplace = False):
        '''
        Input:
          X : 4d array, missing values are -1.
          template : 0-1 3d array, 1 means the voxel is of interest. 
        '''
        if inplace:
            Y = X
        else:
            Y = np.copy(X)
        for ind in tqdm(range(X.shape[0])):
            self.fit_transform_one_img(Y[ind,:,:,:], template, inplace=True)
        return Y
         
        
    def fit_transform_one_img(self, X, template, inplace = False):
        '''
        Input:
          X : 3d array, missing values are -1.
          template : 0-1 3d array, 1 means the voxel is of interest. 
        '''
        long_form = []
        for x in range(X.shape[0]):
            for y in range(X.shape[1]):
                for z in range(X.shape[2]):
                    if template[x, y, z] > 0:
                        long_form.append([x, y, z, X[x, y, z]])
        long_form = np.array(long_form)
        train_ind = long_form[:,3] >= 0
        test_ind = long_form[:,3] < 0
        X_train = long_form[train_ind,:3].astype(int)
        y_train = long_form[train_ind,3]
        X_test = long_form[test_ind, :3].astype(int)
        self.model.fit(X_train, y_train)
        y_test = self.model.predict(X_test)
        if inplace:
            Y = X
        else:
            Y = np.copy(X)
        for ind, (x, y, z) in enumerate(X_test):
            Y[x, y, z] = y_test[ind]
        return Y
        

In [5]:
#preprocess find the high missing rate region
all_missed = (np.mean(data < 0, axis=0) >= .95)
selected = np.logical_not(all_missed)[np.newaxis, :, :, :]

In [ ]:
# fit_model
model = kNN_imputation(n_neighbors=6, weights='distance')
imputed = model.fit_transform(data, selected[0], inplace=True)

 36%|█████████████▉                         | 1552/4345 [09:43<18:27,  2.52it/s]

In [ ]:
# #store_data
# np.savez("../data/imputed_data_kNN_neighbor_6_weights_distance_2022_08_22.npz",imputed=imputed, selected=selected)

In [ ]:
#visualize a specific gene in the gene_index
i = 9
plt.figure()
plt.imshow(np.sum(np.maximum(data[i],0), 0))
plt.colorbar()
plt.figure()
plt.imshow(np.sum(np.maximum(data[i],0), 1))
plt.colorbar()
plt.figure()
plt.imshow(np.sum(np.maximum(data[i],0), 2))
plt.colorbar()
plt.show()

## Holdout test for KNN

In [ ]:
# number of holdouts per gene
holdout_size = 1000

#load_data
tmp = np.load('../data/mouse_brain_ISH_float32.npz',allow_pickle=True)
data = tmp['data']
sections = tmp['sections'].tolist()
original_shape = data.shape

# create test set with holdouts
data_holdout = data.copy()
data_holdout.shape

In [ ]:
# pick random holdout indices and set that index to be imputed (value of -1)

test_index = [] # store each test index
test_actual = np.zeros((data.shape[0]*holdout_size)) # store each actual value

counter = 0

# loop through each gene
for g in range(data.shape[0]):

    # loop through range of holdouts and set values to -1 (skip values that are already -1)
    for a in range(holdout_size):
        index_tmp = [g, np.random.choice(data.shape[1], 1)[0],
                    np.random.choice(data.shape[2], 1)[0], np.random.choice(data.shape[3], 1)[0]]
        while data_holdout[g,index_tmp[1],index_tmp[2],index_tmp[3]] == 0:
            index_tmp = [g, np.random.choice(data.shape[1], 1)[0],
                    np.random.choice(data.shape[2], 1)[0], np.random.choice(data.shape[3], 1)[0]]
        while data_holdout[g,index_tmp[1],index_tmp[2],index_tmp[3]] == -1:
            index_tmp = [g, np.random.choice(data.shape[1], 1)[0],
                    np.random.choice(data.shape[2], 1)[0], np.random.choice(data.shape[3], 1)[0]]
        
        # append to list of test indices
        test_index.append(index_tmp)
        
        # update holdout data
        data_holdout[index_tmp[0], index_tmp[1], index_tmp[2], index_tmp[3]] = -1.
        
        # save actual value 
        test_actual[counter] = data[index_tmp[0], index_tmp[1], index_tmp[2], index_tmp[3]]
        
        counter  += 1

In [ ]:
# run KNN imputation

#preprocess find the high missing rate region
all_missed = (np.mean(data < 0, axis=0) >= .95)
selected = np.logical_not(all_missed)[np.newaxis, :, :, :]

# fit_model
model = kNN_imputation(n_neighbors=6, weights='distance')
imputed_holdout = model.fit_transform(data_holdout, selected[0], inplace=True)

In [ ]:
# identify predicted values from KNN imputation for holdout set

test_predicted = np.zeros((len(test_index))) # store predicted values

counter = 0

# loop through each gene
for g in range(data.shape[0]):

    # loop through range of holdouts to get predicted values
    for a in range(holdout_size):
        
        # store predicted values
        test_predicted[counter] = imputed_holdout[test_index[counter][0],test_index[counter][1],
                                                  test_index[counter][2],test_index[counter][3]]
        counter += 1

In [ ]:
# estimate mean error and standard deviation

error = test_actual - test_predicted
print("Mean error:", round(np.mean(error),6))
print("Standard deviation of error:", round(np.std(error),6))

In [ ]:
from scipy.stats.stats import spearmanr, pearsonr
display(spearmanr(test_predicted,test_actual))
display(pearsonr(test_predicted,test_actual))

## Accuracy improvement from KNN

In [ ]:
# import libraries
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
from pca_with_missing_values import pca_with_missing_values
import pickle
from scipy import ndimage # for calculating Center-of-mass
from scipy.optimize import linear_sum_assignment
import sklearn.preprocessing
from staNMF_ import staNMF
from staNMF.nmf_models import sklearn_nmf

In [ ]:
# load_data

# imputed
tmp = np.load('../data/imputed_data_kNN_neighbor_6_weights_distance.npz')
data = tmp['imputed']
selected = tmp['selected']
original_shape = data.shape

#not imputed
tmp = np.load('../data/mouse_brain_ISH_float32.npz',allow_pickle=True)
data = tmp['data']

In [ ]:
#load reference atlas
areas_atlas = np.load('../data/mouse_coarse_structure_atlas.npy')
mouse_coarse_df = pd.read_pickle('../data/mouse_coarse_df')

In [ ]:
# preprocess compute the support
support = np.sum(areas_atlas, 0) > 0

In [ ]:
# preprocess : get the data within the reference atlas
filtered_data = data[:,:-1,:-1,:-1][:, support]
filtered_data.shape

In [ ]:
###################################
# Create model with 11 PPs, coeffs
###################################

from sklearn_nmf import sklearn_nmf

nmf = sklearn_nmf(n_components=11, l1_ratio=1, alpha = 0, random_state=1)
nmf.fit(np.maximum(filtered_data,0)) # fit model

# Create PPs and coeffs
PPs = nmf.components_
coeffs = nmf.transform(np.maximum(filtered_data,0))
print(PPs.shape, coeffs.shape)

In [ ]:
# function create gene-by-gene reconstruction accuracy

def geneReconstructionAcccuracy(_PPs, _coeffs):

    gene_rec_accuracy_lst = []

    # loop through each gene
    for i in range(len(_coeffs)):

        # reconstruct gene
        gene_rec = np.matmul(_coeffs[i], _PPs)

        # get actual gene from expression data
        gene_actual = np.maximum(filtered_data[i],0)

        # estimate Pearson corr coeff between gene reconstruction and original data
        corr = np.corrcoef(gene_rec,gene_actual)[0][1]

        # set to 0 if NaN
        if np.isnan(corr):
            corr = 0
        
        # add calculated correlation to list
        gene_rec_accuracy_lst.append(corr)        
    
    return gene_rec_accuracy_lst

In [ ]:
# Run reconstruction function for for DecGene
gene_rec_accuracy_DG = geneReconstructionAcccuracy(_PPs=PPs, _coeffs=coeffs)

In [ ]:
# Comparing mean accuracy and standard deviation
print("DecGene accuracy & std:",
      round(np.mean(gene_rec_accuracy_DG),3),
      ", std:",
      round(np.std(gene_rec_accuracy_DG),3))